 - Needs a Google API key
 - [Create credentials and give access to Places APIs](https://console.cloud.google.com/google/maps-apis/credentials)
 - nbextension just shows maps in notebook, could add pins and stuff

```
conda install -c conda-forge -y gmaps
conda install -c conda-forge -y jupyter_contrib_nbextensions
jupyter nbextension enable --py gmaps
jupyter notebook
```


In [1]:
import time
import pdb

import requests, json 

import gmaps
with open('apikey.txt') as f:
    api_key = f.readline().strip()
    f.close

gmaps.configure(api_key=api_key)



In [2]:
new_york_coordinates = (40.6782308, -73.9974274)
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(center=new_york_coordinates, zoom_level=12, layout=figure_layout)


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [3]:
ratings = {}
addresses = {}
locations = {}
nratings = {}
URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
MIN_USER_RATINGS = 20
MIN_RATING = 4

# home
mylocation = '40.6782308,-73.9974274'
# grand army plaza
# location = '40.671872,-73.972544'
# union st and 4th ave
# location ='40.677485,-73.983310'
# bay ridge
# location = "40.624468,-74.0487134"

#rankby='prominence'
rankby='distance'
keyword='pizza'
keyword='coffee'

ltype='establishment'
radius='3000'
#https://developers.google.com/places/web-service/supported_types

In [4]:
def get_gmaps_results(api_key, location, **kwargs):

    request_url = URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + val
    r = requests.get(request_url)
    j = r.json()
    return j

def get_next_page(api_key, next_page_token):
    r = requests.get(URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j

def process_results(results):
    global ratings, addresses, locations, nratings
    for i, res in enumerate(results): 
        if res['user_ratings_total'] < MIN_USER_RATINGS:
            continue
        if res['rating'] < MIN_RATING:
            continue
        addresses[res['name']] = res['vicinity']
        ratings[res['name']] = res['rating']
        nratings[res['name']] = res['user_ratings_total']
        locations[res['name']] = res['geometry']['location']
        print(res['name'], res['rating'], res['geometry']['location'])
    return len(results)+1

In [5]:
def runquery(api_key, location, **kwargs):
    j = get_gmaps_results(api_key, location, **kwargs)
    results = j['results']
    totalresults = 0
    
    totalresults += process_results(results)
    
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = get_next_page(api_key, next_page_token)
        results = j['results'] 
        totalresults += process_results(results)
        
    return totalresults

In [6]:
# use either rankby or radius
# j = get_gmaps_results(api_key, location, keyword=keyword, ltype=ltype, radius=radius)
runquery(api_key, mylocation, keyword=keyword, ltype=ltype, rankby=rankby)
# runquery(api_key, location, keyword=keyword, ltype=ltype, radius=radius)


Seeds of love 4.1 {'lat': 40.6788988, 'lng': -73.9973881}
Brooklyn Habit 4.6 {'lat': 40.67799240000001, 'lng': -73.9960648}
Nerd be cool. Espresso Lab 4.6 {'lat': 40.67619490000001, 'lng': -73.99912929999999}
East One Coffee Roasters 4.5 {'lat': 40.6811876, 'lng': -73.9966204}
Bond St Coffee and Goods 4.7 {'lat': 40.67797669999999, 'lng': -73.9908619}
Cobble Hill Coffeeshop 4.2 {'lat': 40.6834534, 'lng': -73.9955585}
Stone Street Coffee Company 4 {'lat': 40.6725607, 'lng': -73.9927822}
Abbotsford Road Coffee Specialists 4.7 {'lat': 40.6793462, 'lng': -73.9853789}
Starbucks 4.1 {'lat': 40.686221, 'lng': -73.99095400000002}
Starbucks 4.3 {'lat': 40.6693374, 'lng': -73.989842}
Blue Bottle Coffee 4.5 {'lat': 40.68735119999999, 'lng': -73.9897502}
Starbucks 4 {'lat': 40.688556, 'lng': -73.99274799999999}
Starbucks 4.4 {'lat': 40.676998, 'lng': -73.983038}
Stumptown Coffee Roasters 4.7 {'lat': 40.6888305, 'lng': -73.9921739}
Ninth Street Espresso 4.5 {'lat': 40.6796286, 'lng': -73.9822585}
P

63

In [7]:
ratings

{'Seeds of love': 4.1,
 'Brooklyn Habit': 4.6,
 'Nerd be cool. Espresso Lab': 4.6,
 'East One Coffee Roasters': 4.5,
 'Bond St Coffee and Goods': 4.7,
 'Cobble Hill Coffeeshop': 4.2,
 'Stone Street Coffee Company': 4,
 'Abbotsford Road Coffee Specialists': 4.7,
 'Starbucks': 4,
 'Blue Bottle Coffee': 4.4,
 'Stumptown Coffee Roasters': 4.7,
 'Ninth Street Espresso': 4.5,
 'Postmark Cafe': 4.7,
 'Absolute Coffee': 4.4,
 'Crumbs & Coffee': 4.4,
 'Kos Kaffe': 4.2,
 'WHITE NOISE COFFEE CO.': 4.5,
 'Devoción': 4.4,
 'Kaigo Coffee Room': 4.8,
 'Sweetwaters Coffee & Tea 5th Ave. Park Slope': 4.6,
 'Hedgehog Coffee': 4.7,
 'Gregorys Coffee': 4.4,
 'Casella Bagel Coffee Shop': 4.3,
 'Gorilla Coffee': 4.4,
 'Java Joe Coffee Tea & Spice Merchant': 4.7,
 'Coffee Project NY': 4.8,
 'Hungry Ghost Coffee': 4.7,
 'Hungry Ghost': 4.4,
 'Konditori': 4.2,
 'Panera Bread': 4,
 'Roots Cafe': 4.7,
 'Cafe Grumpy - Park Slope': 4.5,
 'Southside Coffee': 4.5,
 'Joe Coffee Company': 4.4,
 'Bluestone Lane DUMBO C

In [8]:
# sort by rating
jarray = []
for name, rating in reversed(sorted(ratings.items(), key=lambda item: item[1])):
    # print (rating, name, addresses[name])
    jarray.append([name, addresses[name], rating, locations[name]['lat'], locations[name]['lng'], nratings[name]])

for name, address, rating, lat, lng, nreviews in jarray:
    print(name, address, rating, lat, lng, nreviews)

Coffee Project NY 78 Rockwell Pl, Brooklyn 4.8 40.68777619999999 -73.9793293 119
Kaigo Coffee Room 139 Bridge Park Dr, Brooklyn 4.8 40.6934911 -74.0009937 81
Roots Cafe 639 5th Ave, Brooklyn 4.7 40.6628539 -73.9916653 252
Hungry Ghost Coffee 80 Dekalb Ave, Brooklyn 4.7 40.6897749 -73.98014979999999 60
Java Joe Coffee Tea & Spice Merchant 414 8th St, Brooklyn 4.7 40.6672891 -73.9808018 59
Hedgehog Coffee 240 7th Ave, Brooklyn 4.7 40.6697216 -73.9794446 32
Postmark Cafe 326 6th St, Brooklyn 4.7 40.6710613 -73.9851996 180
Stumptown Coffee Roasters 212b Pacific St, Brooklyn 4.7 40.6888305 -73.9921739 187
Abbotsford Road Coffee Specialists 573 Sackett St, Brooklyn 4.7 40.6793462 -73.9853789 111
Bond St Coffee and Goods 365 Bond St, Brooklyn 4.7 40.67797669999999 -73.9908619 41
Sweetwaters Coffee & Tea 5th Ave. Park Slope 55 5th Ave, Brooklyn 4.6 40.68102620000001 -73.976986 55
Nerd be cool. Espresso Lab 518 Court St, Brooklyn 4.6 40.67619490000001 -73.99912929999999 62
Brooklyn Habit 405 Sm

In [9]:
markers = [(lat, lng) for name, address, rating, lat, lng, nreviews in jarray]

marker_dict = [{'name': name, 'address': address, 'rating': rating, 'nratings': nreviews} for 
               name, address, rating, lat, lng, nreviews in jarray]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Rating</dt><dd>{rating}</dd>
<dt>Reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**d_item) for d_item in marker_dict]

marker_layer = gmaps.marker_layer(markers, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=eval(mylocation), zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))